### Q1.1

Implement your hash functions by scratch, no ready-made hash functions are allowed. 

# Q1: Hashing

For this task, we are dealing with hashing algorithms. In particular you will implement hash functions and an algorithm called HyperLogLog (HLL).

There are many scenarios where we need to know (or at least estimate) the cardinality of a dataset, e.g., statistical purposes. Consider the need to determine the number of distinct people visiting a website. For famous social media (e.g., Facebook, Instagram) or e-commerce sites (e.g., Amazon, ASOS), this task could be computationally expensive because of the large number of users. Doing this with traditional methods (e.g., SQL query) on a dataset as massive as the sites mentioned above could take days and large amounts and memory. Is it not a better idea to approximately estimate the number of distinct users? HyperLogLog is an algorithm that allows us to make decent guesses when counting vast numbers of distinct elements, with very little computation or memory required.

Q1.1

Implement your hash functions by scratch, no ready-made hash functions are allowed.

In [1]:
import pandas as pd
import numpy as np
import csv
import re
import textwrap
from collections import defaultdict
from math import sqrt
import time
import math

In [2]:
examples = []
count = 0

with open('hash.txt', 'r') as data:
    for line in data:
        if count<10:
            examples.append(line)
            count +=1
        else:
            break

examples = [i[:-1] for i in examples]
print(examples)

['844082e02a27ddee8d99ea1af94a2969', 'ff96d6665b5c59d3a70bb8f2ba4f10be', 'b64a85884e2b159829331c19e05dbac9', '1c8836719e84867c26ba2cfeb372c53d', 'b66f73ffd9008d9c99159e164261df51', '5f414364fde0ca5a72b5a4313d5266aa', 'fdcb0c09722ea060d81cba08681a29b9', '0ebca291ec5fa3f2146eb1c3a219fe13', '1ed24bee346c4c34a491a3db6ce41e73', '4d6289943c24816060240ddff3c530d7']


In [3]:
np.random.seed(123)
# take one coefficient for each slice of the string
a = []
for i in range(5):
    a.append(np.random.randint(0, 65536))

def hash_fun(string):

    # divide the string into 5 parts. 
    slices = textwrap.wrap(string, 6)
    slices[4] = slices[4] + slices[-1] # Since the strings' length is 31, we need to add the last element to the 5th slice
    slices.remove(slices[-1])

    # turn everything into a number and sum up the elements of each part
    x = []
    for slce in slices:
        x.append(sum([ord(i) if i.isalpha() else int(i) for i in slce]))

  # multiply by the coefficients, sum up and take the module
    return '{0:014b}'.format(sum([a[i]*x[i] for i in range(5)]) % 65536)

In [4]:
hashed_value = hash_fun(examples[7])
bits = len(hash_fun(examples[2]))
N = 2**bits - 3 

print(hashed_value)
print(bits)
print(N) #Number of Buckets

100110110110101
15
32765


Please note that the number of buckets we have is linked to the bits length of the buckets index. In particular, the largest bit length we can have as an output of the hash function is 14, and the number of buckets corresponds exactly to 2^{largest_bit_length}+1.

### Q1.2

Use your hash function, implement a HyperLogLog structure. Read the dataset sequentially and add it to your HyperLogLog. At the end you have to provide:
* The cardinality of the dataset.
* The error of your filter.

In [5]:
b = 11
m = 2**11

int(hash_fun(examples[0])[0:b+1])+1

100100010011

In [3]:
registers = defaultdict(int)
for i in range(m):
    registers[i] = -np.inf


with open('hash.txt', 'r') as data:
    for line in data:
        x = hash_fun(line)
        j = 1 + x[0:b+1]
        w = np.prod([int(i) for i in x[b+1:]])